In [50]:
import pyspark.sql.functions as F
import os
from dotenv import load_dotenv
import os

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Spark project')\
        .config('spark.driver.extraClassPath','/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.6.0.jar')\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/13 10:48:05 WARN Utils: Your hostname, LAPTOP-GRLQJBNM resolves to a loopback address: 127.0.1.1; using 172.27.87.196 instead (on interface eth0)
23/09/13 10:48:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/13 10:48:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/13 10:48:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Defining path to the dataset
customer_data_path = "./Data/olist_customers_dataset.csv"
order_item_path = "./Data/olist_order_items_dataset.csv"
order_payment_path = "./Data/olist_order_payments_dataset.csv"
product_category_translation_path= "./Data/product_category_name_translation.csv"
product_path = './Data/olist_products_dataset.csv'
seller_path = './Data/olist_sellers_dataset.csv'
geolocation_path = './Data/olist_geolocation_dataset.csv'
orders_path = './Data/olist_orders_dataset.csv'
reviews = './Data/olist_order_reviews_dataset.csv'

# Reading the datasets
reviews_df = spark.read.csv(reviews, header=True, inferSchema=True)
customer_df = spark.read.csv(customer_data_path, header=True, inferSchema=True)
order_item_df = spark.read.csv(order_item_path, header=True, inferSchema=True)
order_payment_df = spark.read.csv(order_payment_path, header=True, inferSchema=True)
product_category_translation_df = spark.read.csv(product_category_translation_path, header=True, inferSchema=True)
seller_df_uncleaned = spark.read.csv(seller_path, header=True, inferSchema=True)
product_df_uncleaned = spark.read.csv(product_path, header=True, inferSchema=True)
geoloacation_df_uncleaned = spark.read.csv(geolocation_path, header=True, inferSchema= True)
orders_df_uncleaned = spark.read.csv(orders_path, header=True, inferSchema= True)

In [23]:
# Remove leading and trailing whitespace from all columns
seller_df_uncleaned.select([F.trim(F.col(c)).alias(c) for c in seller_df_uncleaned.columns])

# Remove whitespace characters between words in all columns
seller_df = seller_df_uncleaned.select([F.regexp_replace(F.col(c), r'\s+', ' ').alias(c) for c in seller_df_uncleaned.columns])

# Orders dataframe

In [27]:
# harek order ko count check gareko.
status_counts = orders_df_uncleaned.groupBy("order_status").count()

# unavailable status vako order lai filter gareko
unavailableDrop = orders_df_uncleaned.filter(status_counts["order_status"] != "unavailable")

check = unavailableDrop.groupBy("order_status").count()


+------------+-----+
|order_status|count|
+------------+-----+
|     shipped| 1107|
|    canceled|  625|
|    invoiced|  314|
|     created|    5|
|   delivered|96478|
|  processing|  301|
|    approved|    2|
+------------+-----+



In [28]:

columns_to_check = ["order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date"]

grouped_null_counts = unavailableDrop.groupBy("order_status").agg(
    *[F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_null_count") for c in columns_to_check]
)


+------------+----------------------------+---------------------------------------+----------------------------------------+
|order_status|order_approved_at_null_count|order_delivered_carrier_date_null_count|order_delivered_customer_date_null_count|
+------------+----------------------------+---------------------------------------+----------------------------------------+
|     shipped|                           0|                                      0|                                    1107|
|    canceled|                         141|                                    550|                                     619|
|    invoiced|                           0|                                    314|                                     314|
|     created|                           5|                                      5|                                       5|
|   delivered|                          14|                                      2|                                       8|


In [29]:
delivered_drop = unavailableDrop.filter(~(F.col("order_status") == "delivered") | F.col("order_approved_at").isNotNull() |  F.col("order_delivered_carrier_date").isNotNull() | F.col("order_delivered_customer_date").isNotNull())

columns_to_check = ["order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date"]

grouped_null_counts = delivered_drop.groupBy("order_status").agg(
    *[F.sum(F.col(c).isNull().cast("int")).alias(f"{c}_null_count") for c in columns_to_check]
)



98832

In [30]:
orders_df = delivered_drop

In [31]:
# orders_df.write.parquet(f"{parquetFilePath}/orders.parquet", mode="overwrite")

### Replacing column on product dataset with content from product category translation dataset

In [32]:
# product ko name lai english ma translate gareko.
product_joined_df= product_df_uncleaned.join(product_category_translation_df, "Product_category_name", "left")

product_df = product_joined_df.drop("product_category_name")

product_df = product_df.withColumnRenamed("product_category_name_english", "product_category_name")



+--------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+--------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|                 40|                       287|                 1|             225|               16|               10|              14|            perfumery|
|3aa071139cb16b67c...|                 44|                       276|                 1|            1000|               30|               18|              20|                  art|
|96bd76ec8810374ed...|                 46|                       250|                 1|       

In [33]:
product_df = product_df.drop(F.col("product_name_lenght"),F.col("product_description_lenght"),F.col("product_photos_qty"))


+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|             225|               16|               10|              14|            perfumery|
|3aa071139cb16b67c...|            1000|               30|               18|              20|                  art|
|96bd76ec8810374ed...|             154|               18|                9|              15|       sports_leisure|
|cef67bcfe19066a93...|             371|               26|                4|              26|                 baby|
|9dc1a7de274444849...|             625|               20|               17|              13|           housewares|
|41d3672d4792049fa...|             200|               38|                5|     

In [34]:
product_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in product_df.columns]).show()

+----------+----------------+-----------------+-----------------+----------------+---------------------+
|product_id|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+----------+----------------+-----------------+-----------------+----------------+---------------------+
|         0|               2|                2|                2|               2|                  623|
+----------+----------------+-----------------+-----------------+----------------+---------------------+



In [35]:
product_df.count()

32951

In [36]:
products_df = product_df.filter(~(F.col("product_category_name").isNull()))


+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|             225|               16|               10|              14|            perfumery|
|3aa071139cb16b67c...|            1000|               30|               18|              20|                  art|
|96bd76ec8810374ed...|             154|               18|                9|              15|       sports_leisure|
|cef67bcfe19066a93...|             371|               26|                4|              26|                 baby|
|9dc1a7de274444849...|             625|               20|               17|              13|           housewares|
|41d3672d4792049fa...|             200|               38|                5|     

In [37]:
products_count =  products_df.groupBy(F.col("product_category_name")).count()


+---------------------+-----+
|product_category_name|count|
+---------------------+-----+
|                  art|   55|
|              flowers|   14|
|    home_construction|  225|
| fashion_male_clot...|   95|
| kitchen_dining_la...|   94|
|     small_appliances|  231|
|           la_cuisine|   10|
|       bed_bath_table| 3029|
| signaling_and_sec...|   93|
|     office_furniture|  309|
|            computers|   30|
|        watches_gifts| 1329|
|                 auto| 1900|
| fashion_bags_acce...|  849|
| construction_tool...|   78|
|           cool_stuff|  789|
|    cds_dvds_musicals|    1|
|                 food|   82|
| computers_accesso...| 1639|
|            perfumery|  868|
+---------------------+-----+
only showing top 20 rows



In [38]:
products_df.count()

32328

In [39]:
products_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in products_df.columns]).show()

+----------+----------------+-----------------+-----------------+----------------+---------------------+
|product_id|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+----------+----------------+-----------------+-----------------+----------------+---------------------+
|         0|               1|                1|                1|               1|                    0|
+----------+----------------+-----------------+-----------------+----------------+---------------------+



In [43]:
product_df = products_df.filter(~F.col('product_weight_g').isNull())


+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+--------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|             225|               16|               10|              14|            perfumery|
|3aa071139cb16b67c...|            1000|               30|               18|              20|                  art|
|96bd76ec8810374ed...|             154|               18|                9|              15|       sports_leisure|
|cef67bcfe19066a93...|             371|               26|                4|              26|                 baby|
|9dc1a7de274444849...|             625|               20|               17|              13|           housewares|
|41d3672d4792049fa...|             200|               38|                5|     

In [40]:
product_df.count()

32951

In [41]:
# product_df.write.parquet(f"{parquetFilePath}/product.parquet", mode="overwrite")

### Geolocation table

In [44]:
geoloacation_df_uncleaned.select('geolocation_zip_code_prefix').groupBy(F.col('geolocation_zip_code_prefix')).agg(F.count(F.col('geolocation_zip_code_prefix'))).show()

+---------------------------+----------------------------------+
|geolocation_zip_code_prefix|count(geolocation_zip_code_prefix)|
+---------------------------+----------------------------------+
|                       1238|                               164|
|                       2122|                                33|
|                       2142|                                 5|
|                       2366|                                33|
|                       2866|                                41|
|                       3175|                                32|
|                       3918|                                50|
|                       4101|                                72|
|                       4935|                                15|
|                       5518|                                27|
|                       1025|                                15|
|                       1127|                                40|
|                       1

In [45]:
geolocation_avg_df = geoloacation_df_uncleaned\
    .groupBy(F.col('geolocation_zip_code_prefix'))\
    .agg(F.avg(F.col('geolocation_lat')).alias('geolocation_lat'), F.avg(F.col('geolocation_lng')).alias('geolocation_lng'))\
    .orderBy(F.col('geolocation_zip_code_prefix'))

In [47]:
geolocation_df = geolocation_avg_df

In [49]:
# geolocation_df.write.parquet(f"{parquetFilePath}/geolocation.parquet", mode="overwrite")

### saving data into postgresql

In [51]:
load_dotenv()

Pg_user = os.getenv("POSTGRES_USER")
Pg_password = os.getenv("POSTGRES_PASSWORD")

In [52]:
customer_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'customer_df', user=Pg_user,password=Pg_password).mode('overwrite').save()


In [ ]:
customer_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'customer_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

order_item_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'order_item_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

order_payment_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'order_payment_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

seller_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'seller_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

orders_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'orders_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

product_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'product_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

geolocation_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',driver = 'org.postgresql.Driver', dbtable = 'geolocation_df', user=Pg_user,password=Pg_password).mode('overwrite').save()

### Reading data from beaver